In [6]:
import json
import random
import os
from collections import defaultdict

def split_coco_dataset(json_path, output_dir, split_ratio=0.95):
    # json 파일 로드 # json 파일 로드
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    # 각 클래스별로 이미지를 추적하기 위한 딕셔너리 생성
    class_to_images = defaultdict(set)
    image_id_to_annotations = defaultdict(list)
    # 각 어노테이션에서 이미지 ID와 클래스 ID를 매핑
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        category_id = ann['category_id']
        class_to_images[category_id].add(image_id)
        image_id_to_annotations[image_id].append(ann)

    # 분할된 데이터셋을 저장할 리스트
    train_images = []
    val_images = []
    train_annotations = []
    val_annotations = []

    # 각 클래스별로 이미지 리스트를 나누기
    for category_id, image_ids in class_to_images.items():
        image_ids = list(image_ids)
        random.shuffle(image_ids)  # 데이터를 무작위로 섞음
        split_index = int(len(image_ids) * split_ratio)
        
        train_image_ids = image_ids[:split_index]
        val_image_ids = image_ids[split_index:]
        # 이미지 ID를 기준으로 이미지와 어노테이션을 추가
        train_images.extend([img for img in coco_data['images'] if img['id'] in train_image_ids])
        val_images.extend([img for img in coco_data['images'] if img['id'] in val_image_ids])
        train_annotations.extend([ann for img_id in train_image_ids for ann in image_id_to_annotations[img_id]])
        val_annotations.extend([ann for img_id in val_image_ids for ann in image_id_to_annotations[img_id]])

    # 기존의 info, licenses, categories 정보는 그대로 복사
    info = coco_data.get('info', {})
    licenses = coco_data.get('licenses', [])
    categories = coco_data.get('categories', [])

    # 중복된 이미지를 제거 (여러 클래스에 속한 경우)
    train_images = {img['id']: img for img in train_images}.values()
    val_images = {img['id']: img for img in val_images}.values()

    # 어노테이션 ID 중복 방지
    def update_annotation_ids(annotations):
        new_annotations = []
        next_id = 1
        
        for ann in annotations:
            new_ann = ann.copy()
            new_ann['id'] = next_id
            new_annotations.append(new_ann)
            next_id += 1
        
        return new_annotations

    # 어노테이션 ID 업데이트 부분
    train_annotations = update_annotation_ids(train_annotations)
    val_annotations = update_annotation_ids(val_annotations)

    # 어노테이션 ID의 고유성 확인
    print(len(set(ann['id'] for ann in train_annotations)),len(train_annotations))
    print(len(set(ann['id'] for ann in val_annotations)), len(val_annotations))
    assert len(set(ann['id'] for ann in train_annotations)) == len(train_annotations), "Train annotation IDs are not unique"
    assert len(set(ann['id'] for ann in val_annotations)) == len(val_annotations), "Validation annotation IDs are not unique"

    # 새로운 COCO 형식의 json 데이터 생성 (train과 val)
    train_data = {
        'info': info,
        'licenses': licenses,
        'images': list(train_images),
        'annotations': train_annotations,
        'categories': categories
    }

    val_data = {
        'info': info,
        'licenses': licenses,
        'images': list(val_images),
        'annotations': val_annotations,
        'categories': categories
    }

    # 결과를 json 파일로 저장
    os.makedirs(output_dir, exist_ok=True)

    train_output_path = os.path.join(output_dir, 'train_'+str(int(split_ratio*100))+'.json')
    val_output_path = os.path.join(output_dir, 'val_'+str(int(100-split_ratio*100))+'.json')

    with open(train_output_path, 'w') as f:
        json.dump(train_data, f)

    with open(val_output_path, 'w') as f:
        json.dump(val_data, f)

    print(f"Train set: {len(train_images)} images, {len(train_annotations)} annotations")
    print(f"Validation set: {len(val_images)} images, {len(val_annotations)} annotations")

# 사용 예시
split_coco_dataset('../../../dataset/train.json', '../../../dataset', split_ratio=0.8)

58423 58423
14270 14270
Train set: 4337 images, 58423 annotations
Validation set: 1598 images, 14270 annotations


In [5]:
import json

def count_unique_image_ids(json_path):
    # JSON 파일 로드
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    # 고유한 image_id를 저장할 집합
    unique_image_ids = set()

    # images 리스트에서 image_id를 수집
    for image in coco_data['images']:
        unique_image_ids.add(image['id'])

    # 고유한 image_id의 수를 출력
    print(f"Unique image_ids count: {len(unique_image_ids)}")

# 사용 예시
count_unique_image_ids('../../../dataset/train_95.json')

Unique image_ids count: 4760
